In [ ]:
from utils import *
import json
import logging
import matplotlib.pyplot as plt

In [2]:
driver = webdriver.Firefox()

# Initialise Google.
driver.get("https://www.google.com/maps")
driver.find_element(By.CSS_SELECTOR, 'button[aria-label="Reject all"]').click()


In [3]:
with open("data/stations.json", "r") as file:
    stations, lines, areas = json.load(file).values()

In [4]:
with open("data/nodes.json", "r") as file:
    nodes = json.load(file)

In [7]:
with open("data/times.npy", "rb") as file:
    mat = np.load(file)

In [ ]:
# Initialise mat.
# n = len(stations)
# mat = np.zeros((n, n), np.int8) 

In [71]:
line = "metropolitan"
route = "cal_che"
url_dict[line][route] = "https://www.google.com/maps/dir/Chalfont+%26+Latimer,+Station+Approach,+Little+Chalfont,+Amersham+HP7+9PR/''/@51.6874621,-0.6288138,13z/am=t/data=!3m1!4b1!4m14!4m13!1m5!1m1!1s0x4876680ef7d58601:0xc89e6163628b7b6a!2m2!1d-0.5605253!2d51.6680694!1m5!1m1!1s0x48765d0c8f367557:0x8939ee4eec34f5d4!2m2!1d-0.6110651!2d51.7052095!3e3!5m1!1e2?entry=ttu&g_ep=EgoyMDI1MDYxMC4xIKXMDSoASAFQAw%3D%3D"

In [72]:
# Requires user input to access and open travel details.
driver.get(url_dict[line][route])

In [73]:
journey, timestamps = get_journey(driver).values()

Number of stations: 2


In [74]:
# Calculate travel time between stations.
times = []
for i in range(len(timestamps) - 1):
    time: int = max((timestamps[i+1] - timestamps[i]).seconds//60, 1)
    times.append(time)

In [75]:
journey

['Chalfont & Latimer', 'Chesham']

In [76]:
# Update time matrix.
for i in range(len(times)):
    start_ind = stations.index(journey[i])
    end_ind = stations.index(journey[i+1])
    prev_time = mat[start_ind, end_ind]
    if prev_time == 0:
        mat[start_ind, end_ind] = times[i]
    else:
        mat[start_ind, end_ind] = 0.2 * times[i] + 0.8 * prev_time
    mat[end_ind, start_ind] = mat[start_ind, end_ind]
    print(f"{i}. Updated ({start_ind}, {end_ind}) from {prev_time} to {mat[start_ind, end_ind]}.")

0. Updated (41, 45) from 0.0 to 8.0.


In [ ]:
nodes["metropolitan"] = {
    "origin": journey[0],
    "ame_ald": journey
}

In [77]:
nodes[line][route] = journey

In [78]:
with open("data/times.npy", "wb") as file:
    np.save(file, mat)

In [79]:
with open("data/nodes.json", "w") as file:
    data = json.dumps(nodes, indent=4)
    file.write(data)

In [80]:
with open("data/url.json", "w") as file:
    data = json.dumps(url_dict, indent=4)
    file.write(data)

In [81]:
driver.quit()

In [ ]:
start = "South Kensington"
travel_time: float = 45
stations_ind = []
start_ind: int = stations.index(start)
find_stations_in_range(mat, start_ind, travel_time, stations_ind)


In [ ]:
set(areas[i] for i in stations_ind)

In [ ]:
# node_pos = np.zeros(len(stations))

In [10]:
piccadilly_nodes = np.zeros((len(stations), 3))

In [53]:
piccadilly_nodes[stations.index("Heathrow Terminal 5")]

array([14.,  9.,  2.])

In [26]:
for i in range(len(nodes["piccadilly"]["act_hiv"])):
    piccadilly_nodes[stations.index(nodes["piccadilly"]["act_hiv"][i])] = [14,i,0]

In [15]:
circle_nodes[stations.index("Bayswater")]

array([25.,  0.])

In [16]:
with open("circle.npy", "wb") as file:
    np.save(file, circle_nodes)

In [69]:
nodes = np.hstack((nodes, np.zeros((269,2))))

In [ ]:
nodes = np.zeros((len(stations), 2))

In [90]:
nodes[stations.index("Hammersmith")]

array([7., 0., 0., 0.])

In [91]:
nodes[stations.index("Edgware Road")]

array([10.,  0.,  5.,  0.])

In [92]:
nodes[stations.index("Paddington")]

array([10.,  0.,  4.,  0.])

In [83]:
i = 0
for station in journey:
    print(station)
    nodes[stations.index(station)] = [10,0,i,0]
    i -= 1

Earl's Court
West Brompton
Fulham Broadway
Parsons Green
Putney Bridge
East Putney
Southfields
Wimbledon Park
Wimbledon


In [51]:
with open("piccadilly.npy", "wb") as file:
    np.save(file, piccadilly_nodes)

In [54]:
driver.quit()

In [28]:
with open("nodes.json", "w") as file:
    data = json.dumps(nodes, indent=4)
    file.write(data)